In [3]:
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import min, max, upper, trim
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

spark = SparkSession.Builder().appName('map() & flatMap() transformation on RDD').getOrCreate()

In [26]:
df = spark.read.parquet('parquetData/')
df.show()
df.count()

+-------------------+---+----------+---------+--------------------+------+---------------+-------------------+--------------------+----------+---------+--------------------+--------------------+
|  registration_dttm| id|first_name|last_name|               email|gender|     ip_address|                 cc|             country| birthdate|   salary|               title|            comments|
+-------------------+---+----------+---------+--------------------+------+---------------+-------------------+--------------------+----------+---------+--------------------+--------------------+
|2016-02-03 13:25:29|  1|    Amanda|   Jordan|    ajordan0@com.com|Female|    1.197.201.2|   6759521864920116|           Indonesia|  3/8/1971| 49756.53|    Internal Auditor|               1E+02|
|2016-02-03 22:34:03|  2|    Albert|  Freeman|     afreeman1@is.gd|  Male| 218.111.175.34|                   |              Canada| 1/16/1968|150280.17|       Accountant IV|                    |
|2016-02-03 06:39:31|  3|

2000

In [27]:
%%time
df.groupBy('gender').max('salary').show()

+------+-----------+
|gender|max(salary)|
+------+-----------+
|Female|  286592.99|
|  Male|  286388.01|
|      |       NULL|
+------+-----------+

CPU times: user 7.47 ms, sys: 0 ns, total: 7.47 ms
Wall time: 235 ms


In [28]:
df.write.parquet('data/writeToParquet/'
                ,mode = 'overwrite'
                ,partitionBy=['gender','country'])

In [36]:
%%time
df = spark.read.parquet('data/writeToParquet/')
df.count()

CPU times: user 6.72 ms, sys: 0 ns, total: 6.72 ms
Wall time: 1.27 s


2000

In [37]:
%%time
df.groupBy('gender').max('salary').show()

+------+-----------+
|gender|max(salary)|
+------+-----------+
|Female|  286592.99|
|  Male|  286388.01|
|  NULL|       NULL|
+------+-----------+

CPU times: user 4.56 ms, sys: 2.49 ms, total: 7.04 ms
Wall time: 615 ms


In [40]:
df = spark.read.parquet('data/writeToParquet/gender=Female')
# removed the Gender column from the dataset
df.count()
df.show()

+-------------------+---+----------+----------+--------------------+---------------+-------------------+----------+---------+--------------------+--------------------+-------+
|  registration_dttm| id|first_name| last_name|               email|     ip_address|                 cc| birthdate|   salary|               title|            comments|country|
+-------------------+---+----------+----------+--------------------+---------------+-------------------+----------+---------+--------------------+--------------------+-------+
|2016-02-03 06:06:21|  4|    Denise|     Riley|    driley3@gmpg.org|  140.35.109.83|   3576031598965625|  4/8/1997| 90263.05|Senior Cost Accou...|                    |  China|
|2016-02-03 23:34:34| 12|     Alice|     Berry|aberryb@wikipedia...| 246.225.12.189|   4917830851454417| 8/12/1968| 22944.53|    Quality Engineer|                    |  China|
|2016-02-03 16:00:36| 20|   Rebecca|      Bell| rbellj@bandcamp.com|172.215.104.127|                   |          |13725

In [ ]:
spar.stop()